# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
all_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
all_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# District Summary Calculations
# Count school using school_data... 'name' was changed to 'school_name'
school_total = school_data['school_name'].count()

#count student total
student_total = all_data['student_name'].count()

# Total budget from school_data
budget_total = school_data['budget'].sum()

# score averages
avg_math_score = all_data['math_score'].mean()
avg_reading_score = all_data['reading_score'].mean()
overall_avg_pass = ((avg_math_score + avg_reading_score) / 2)

# percentage of students passing
pass_math = len(all_data.loc[all_data['math_score'] > 69])
pass_reading = len(all_data.loc[all_data['reading_score'] > 69])
percent_pass_math = (pass_math / student_total * 100)
percent_pass_reading = (pass_reading / student_total * 100)

In [3]:
# District Summary.DataFrame
district_summary = pd.DataFrame({"Total Schools": [school_total],
                                "Total Students": [student_total],
                                "Total Budget": [budget_total],
                                "Average Math Score": [avg_math_score],
                                "Average Reading Score": [avg_reading_score],
                                "% Passing Math": [percent_pass_math],
                                "% Passing Reading": [percent_pass_reading],
                                "% Overall Passing Rate": [overall_avg_pass]})
district_summary

#formatting
district_summary.style.format({"Total Students": "{:,}",
                               "Total Budget": "${:,.2f}",
                               "Average Math Score": "{:.2f}%",
                               "Average Reading Score": "{:.2f}%",
                               "% Passing Math": "{:.2f}%",
                               "% Passing Reading": "{:.2f}%",
                               "% Overall Passing Rate": "{:.2f}%"})


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.99%,81.88%,74.98%,85.81%,80.43%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [24]:
# School summary grouping per school

school_type = all_data['type'].groupby(all_data['school_name']).unique().map(lambda x: x[0]).rename("School Type")
students_per_school = all_data['size'].groupby(all_data['school_name']).unique().map(lambda x: x[0]).rename("Total Students")
school_budget = all_data['budget'].groupby(all_data['school_name']).unique().map(lambda x: x[0]).rename("School Budget")
budget_per_student = (school_budget / students_per_school).rename("Per Student Budget")

avg_math_per_school = all_data['math_score'].groupby(all_data['school_name']).mean().rename("Average Math Score")
avg_reading_per_school = all_data['reading_score'].groupby(all_data['school_name']).mean().rename("Average Reading Score")

passed_math_per_school = all_data.loc[all_data['math_score'] > 69].groupby(['school_name'])['math_score'].count()
percent_pass_math_school = (passed_math_per_school / students_per_school * 100).rename("% Passing Math")
passed_reading_per_school = all_data.loc[all_data['reading_score'] > 69].groupby(all_data['school_name'])['reading_score'].count()
percent_pass_reading_school = (passed_reading_per_school / students_per_school * 100).rename("% Passing Reading")
overall_passing_school = ((percent_pass_math_school + percent_pass_reading_school) / 2).rename("% Overall Passing Rate")
                                       

In [25]:
# School Summary for each school
school_summary = pd.concat([school_type,
                           students_per_school,
                           school_budget,
                           budget_per_student,
                           avg_math_per_school,
                           avg_reading_per_school,
                           percent_pass_math_school,
                           percent_pass_reading_school,
                           overall_passing_school],
                           axis=1)
school_summary

#Formatting
school_summary.style.format({"Total Students": "{:,}", 
                          "School Budget": "${:,}", 
                          "Per Student Budget": "${:.2f}",
                          "Average Math Score": "{:.2f}%", 
                          "Average Reading Score": "{:.2f}%", 
                          "% Passing Math": "{:.2f}%", 
                          "% Passing Reading": "{:.2f}%", 
                          "% Overall Passing Rate": "{:.2f}%"})


,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628.00,77.05%,81.03%,66.68%,81.93%,74.31%
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06%,83.98%,94.13%,97.04%,95.59%
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71%,81.16%,65.99%,80.74%,73.36%
Ford High School,District,"2,739","$1,763,916",$644.00,77.10%,80.75%,68.31%,79.30%,73.80%
Griffin High School,Charter,"1,468","$917,500",$625.00,83.35%,83.82%,93.39%,97.14%,95.27%
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29%,80.93%,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087",$581.00,83.80%,83.81%,92.51%,96.25%,94.38%
Huang High School,District,"2,917","$1,910,635",$655.00,76.63%,81.18%,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",$650.00,77.07%,80.97%,66.06%,81.22%,73.64%


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [8]:
top_performing_pass = school_summary.sort_values(["% Overall Passing Rate"], ascending=False)

top_performing_pass.head()

top_performing_pass.style.format({"Total Students": "{:,}",
                   "School Budget": "${:,}", 
                   "Per Student Budget": "${:.0f}", 
                   "% Passing Math": "{:.2f}%",
                   "% Passing Reading": "{:.2f}%", 
                   "% Overall Passing Rate": "{:.2f}%"})

,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.0619,83.9758,94.13%,97.04%,95.59%
Thomas High School,Charter,"1,635","$1,043,130",$638,83.4183,83.8489,93.27%,97.31%,95.29%
Pena High School,Charter,962,"$585,858",$609,83.8399,84.0447,94.59%,95.95%,95.27%
Griffin High School,Charter,"1,468","$917,500",$625,83.3515,83.8168,93.39%,97.14%,95.27%
Wilson High School,Charter,"2,283","$1,319,574",$578,83.2742,83.9895,93.87%,96.54%,95.20%
Wright High School,Charter,"1,800","$1,049,400",$583,83.6822,83.955,93.33%,96.61%,94.97%
Shelton High School,Charter,"1,761","$1,056,600",$600,83.3595,83.7257,93.87%,95.85%,94.86%
Holden High School,Charter,427,"$248,087",$581,83.8033,83.815,92.51%,96.25%,94.38%
Bailey High School,District,"4,976","$3,124,928",$628,77.0484,81.034,66.68%,81.93%,74.31%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [9]:
bottom_performing_pass = school_summary.sort_values(["% Overall Passing Rate"], ascending=True)
bottom_performing_pass.head()
bottom_performing_pass.style.format({"Total Students": "{:,}",
                                     "School Budget": "${:,}", 
                                     "Per Student Budget": "${:.0f}", 
                                     "% Passing Math": "{:.2f}%",
                                     "% Passing Reading": "{:.2f}%",
                                     "% Overall Passing Rate": "{:.2f}%"})


,School Type,Total Students,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637,76.8427,80.7447,66.37%,80.22%,73.29%
Figueroa High School,District,"2,949","$1,884,411",$639,76.7118,81.158,65.99%,80.74%,73.36%
Huang High School,District,"2,917","$1,910,635",$655,76.6294,81.1827,65.68%,81.32%,73.50%
Johnson High School,District,"4,761","$3,094,650",$650,77.0725,80.9664,66.06%,81.22%,73.64%
Ford High School,District,"2,739","$1,763,916",$644,77.1026,80.7463,68.31%,79.30%,73.80%
Hernandez High School,District,"4,635","$3,022,020",$652,77.2898,80.9344,66.75%,80.86%,73.81%
Bailey High School,District,"4,976","$3,124,928",$628,77.0484,81.034,66.68%,81.93%,74.31%
Holden High School,Charter,427,"$248,087",$581,83.8033,83.815,92.51%,96.25%,94.38%
Shelton High School,Charter,"1,761","$1,056,600",$600,83.3595,83.7257,93.87%,95.85%,94.86%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [10]:
school_grade_math = all_data.groupby(['school_name', 'grade'])['math_score'].mean().unstack()
school_grade_math.reindex(['9th','10th','11th', '12th'], axis=1)

school_grade_math.style.format({"9th": "{:.2f}%",
                                "10th": "{:.2f}%", 
                                "11th": "{:.2f}%", 
                                "12th": "{:.2f}%"})

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,77.00%,77.52%,76.49%,77.08%
Cabrera High School,83.15%,82.77%,83.28%,83.09%
Figueroa High School,76.54%,76.88%,77.15%,76.40%
Ford High School,77.67%,76.92%,76.18%,77.36%
Griffin High School,84.23%,83.84%,83.36%,82.04%
Hernandez High School,77.34%,77.14%,77.19%,77.44%
Holden High School,83.43%,85.00%,82.86%,83.79%
Huang High School,75.91%,76.45%,77.23%,77.03%
Johnson High School,76.69%,77.49%,76.86%,77.19%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [11]:
school_grade_reading = all_data.groupby(['school_name', 'grade'])['reading_score'].mean().unstack()
school_grade_reading.reindex(['9th','10th','11th', '12th'], axis=1)

school_grade_reading.style.format({"9th": "{:.2f}%",
                                "10th": "{:.2f}%", 
                                "11th": "{:.2f}%", 
                                "12th": "{:.2f}%"})

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,80.91%,80.95%,80.91%,81.30%
Cabrera High School,84.25%,83.79%,84.29%,83.68%
Figueroa High School,81.41%,80.64%,81.38%,81.20%
Ford High School,81.26%,80.40%,80.66%,80.63%
Griffin High School,83.71%,84.29%,84.01%,83.37%
Hernandez High School,80.66%,81.40%,80.86%,80.87%
Holden High School,83.32%,83.82%,84.70%,83.68%
Huang High School,81.51%,81.42%,80.31%,81.29%
Johnson High School,80.77%,80.62%,81.23%,81.26%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [12]:
# I'm using the sample bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
budget_bins = pd.cut(school_summary["Per Student Budget"], spending_bins, labels=group_names)
budget_stats = all_data.join(budget_bins, on="school_name")
students_per_budget = budget_stats["size"].groupby(budget_stats["Per Student Budget"]).count()
pass_math_budget = budget_stats.loc[budget_stats['math_score'] > 69].groupby(['Per Student Budget'])['math_score'].count()
pass_reading_budget = budget_stats.loc[budget_stats['reading_score'] > 69].groupby(['Per Student Budget'])['reading_score'].count()
avg_math_budget = budget_stats['math_score'].groupby(budget_stats['Per Student Budget']).mean().rename("Average Math Score")
avg_reading_budget = budget_stats['reading_score'].groupby(budget_stats['Per Student Budget']).mean().rename("Average Reading Score")
percent_pass_math_budget = (pass_math_budget / students_per_budget * 100).rename("% Passing Math")
percent_pass_reading_budget = (pass_reading_budget / students_per_budget * 100).rename("% Passing Reading")
overall_pass_budget = ((percent_pass_math_budget + percent_pass_reading_budget) / 2).rename("% Overall Passing Rate")


In [13]:
budget_summary = pd.concat([avg_math_budget,
                            avg_reading_budget,
                            percent_pass_math_budget,
                            percent_pass_reading_budget,
                            overall_pass_budget],
                            axis=1)
budget_summary

budget_summary.style.format({"Average Math Score": "{:.2f}%",
                            "Average Reading Score": "{:.2f}%",
                            "% Passing Math": "{:.2f}%",
                            "% Passing Reading": "{:.2f}%",
                            "% Overall Passing Rate": "{:.2f}%"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Per Student Budget,,,,,
<$585,83.36%,83.96%,93.70%,96.69%,95.19%
$585-615,83.53%,83.84%,94.12%,95.89%,95.01%
$615-645,78.06%,81.43%,71.40%,83.61%,77.51%
$645-675,77.05%,81.01%,66.23%,81.11%,73.67%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [14]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
size_bins = pd.cut(all_data["size"], size_bins, labels=group_names)
size_stats = all_data.join(size_bins, lsuffix="actual")
students_per_size = size_stats["size"].groupby(size_stats["size"]).count()
pass_math_size = size_stats.loc[size_stats['math_score'] > 69].groupby(['size'])['math_score'].count()
pass_reading_size = size_stats.loc[size_stats['reading_score'] > 69].groupby(['size'])['reading_score'].count()
avg_math_size = size_stats['math_score'].groupby(size_stats['size']).mean().rename("Average Math Score")
avg_reading_size = size_stats['reading_score'].groupby(size_stats['size']).mean().rename("Average Reading Score")
percent_pass_math_size = (pass_math_size / students_per_size * 100).rename("% Passing Math")
percent_pass_reading_size = (pass_reading_size / students_per_size * 100).rename("% Passing Reading")
overall_pass_size = ((percent_pass_math_size + percent_pass_reading_size) / 2).rename("% Overall Passing Rate")


In [15]:
size_summary = pd.concat([avg_math_size, 
                            avg_reading_size, 
                            percent_pass_math_size, 
                            percent_pass_reading_size, 
                            overall_pass_size],  
                            axis=1)
size_summary

size_summary.style.format({"Average Math Score": "{:.2f}%",
                          "Average Reading Score": "{:.2f}%",
                          "% Passing Math": "{:.2f}%",
                          "% Passing Reading": "{:.2f}%",
                          "% Overall Passing Rate": "{:.2f}%"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
size,,,,,
Small (<1000),83.83%,83.97%,93.95%,96.04%,95.00%
Medium (1000-2000),83.37%,83.87%,93.62%,96.77%,95.19%
Large (2000-5000),77.48%,81.20%,68.65%,82.13%,75.39%


## Scores by School Type

* Perform the same operations as above, based on school type.

In [16]:
students_per_type = all_data["type"].groupby(all_data["type"]).count()
pass_math_type = all_data.loc[all_data['math_score'] > 69].groupby(['type'])['math_score'].count()
pass_reading_type = all_data.loc[all_data['reading_score'] > 69].groupby(['type'])['reading_score'].count()
avg_math_type = all_data['math_score'].groupby(all_data['type']).mean().rename("Average Math Score")
avg_reading_type = all_data['reading_score'].groupby(all_data['type']).mean().rename("Average Reading Score")
perc_pass_math_type = (pass_math_type / students_per_type * 100).rename("% Passing Math")
perc_pass_reading_type = (pass_reading_type / students_per_type * 100).rename("% Passing Reading")
overall_pass_type = ((perc_pass_math_type + perc_pass_reading_type) / 2).rename("% Overall Passing Rate")

In [17]:
type_summary = pd.concat([avg_math_type, 
                            avg_reading_type, 
                            perc_pass_math_type, 
                            perc_pass_reading_type, 
                            overall_pass_type],  
                            axis=1)
type_summary

type_summary.style.format({"Average Math Score": "{:.2f}%", 
                              "Average Reading Score": "{:.2f}%", 
                              "% Passing Math": "{:.2f}%", 
                              "% Passing Reading":"{:.2f}%", 
                              "% Overall Passing Rate": "{:.2f}%"})

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
type,,,,,
Charter,83.41%,83.90%,93.70%,96.65%,95.17%
District,76.99%,80.96%,66.52%,80.91%,73.71%
